## Imports

In [1]:
import networkx as nx
import os
import sys

# Make the src/ directory available
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from planners.IPBasicPRM import BasicPRM
from optimize_path import OptimizeFlyby
from collision_checker import CollisionChecker
from benchmarks import construct_benchmark_environments
from interactive_widgets import (
    interactive_environment_exploration,
    interactive_radius_exploration,
    interactive_k_exploration,
    interactive_global_k_optimization,
    interactive_individual_k_optimization
)

/home/nico/workspace/github.com/Nico-Sander/robotprogramming-project-2526/src/collision_checker.py:273: SyntaxWarning: invalid escape sequence '\_'
  It does not rely on a simple list of points; instead, it reads the geometric parameters ($r$, $fixed\_k$, $P_{2n}$)


## Load Environments

In [2]:
env_dict = construct_benchmark_environments()

## Create Planner and Collision Checker for each environment

In [3]:
for name, item in env_dict.items():
    # 1. Initialize a CollisionChecker with the environment
    cc = CollisionChecker(item["env"])

    # 2. Initialize a Planner with the collisionChecker
    planner = BasicPRM(cc)

    # 3. Construct the NetworkX Graph from the predefined solution path
    G = nx.Graph()

    node_names = []

    for i, coord in enumerate(item["smooth_path"]):
        # Determine the name based on position in the list
        if i == 0:
            n = "start"
        elif i == len(item["smooth_path"]) - 1:
            n = "goal"
        else:
            n = f"{i}" # Naming middle nodes generically
        
        node_names.append(n)
        
        # Add node with the 'pos' attribute
        G.add_node(n, pos=coord)

    # Add edges connecting each node to the next
    for i in range(len(node_names) - 1):
        current_node = node_names[i]
        next_node = node_names[i+1]
        G.add_edge(current_node, next_node)

    # 4. Inject the constructed graph into the Planner instance
    planner.graph = G
    
    # 5. Add the node names and the planner to the environment dictionary
    env_dict[name]["solution_node_names"] = node_names
    env_dict[name]["planner"] = planner
    del env_dict[name]["smooth_path"]

# pprint.pprint(env_dict)

## Show the Environments

In [4]:
interactive_environment_exploration(env_dict)

## Interactive Radius Exploration (k = symmetric)

In [5]:
optimizer = OptimizeFlyby()

interactive_radius_exploration(env_dict, optimizer)

## Interactive k (Skew) Exploration (r = fixed)

In [6]:
optimizer = OptimizeFlyby()

interactive_k_exploration(env_dict, optimizer, r_fixed=0.5)

## Optimize Global k (Skew)

In [7]:
optimizer = OptimizeFlyby()

interactive_global_k_optimization(env_dict, optimizer)

Dropdown(description='Select Env:', options=('1', '2', '3', '4'), value='1')

## Optimize Individual k's (Skew)

In [8]:
optimizer = OptimizeFlyby()

interactive_individual_k_optimization(env_dict, optimizer)

Dropdown(description='Select Env:', options=('1', '2', '3', '4'), value='1')